In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from pymongo import MongoClient
import os
from dotenv import load_dotenv

# Load env variables
load_dotenv()
mongo_uri = os.getenv("MONGO_URI")
db_name = os.getenv("MONGO_DB")
collection_name = os.getenv("MONGO_COLLECTION")

# Connect to MongoDB and load data
client = MongoClient(mongo_uri)
db = client[db_name]
collection = db[collection_name]
data = list(collection.find())

# Convert to DataFrame
df = pd.DataFrame(data)

# Fix ObjectId and datetime fields
if "_id" in df.columns:
    df["_id"] = df["_id"].astype(str)
df["lead_created_at"] = pd.to_datetime(df["lead_created_at"], errors="coerce")

In [2]:
df.head()

,_id,lead_id,email,first_name,last_name,created_at,input_channel,campaign_id,name,started_at,...,created_at_insc,amount,lead_created_at,inscription_created_at,cost_float,amount_float,conversion_time_days,converted,conversion_valid,conversion_anomaly_type
0,6867947d37953a251ea5eea4,9e5068ff-c01f-4eab-bb38-214fb9d6b83d,ktytcomb0@sina.com.cn,Kristan,,2025-05-28,WEB_ORGANIC,None,None,NaT,...,2025-04-26,"1841,57 €",2025-05-28,2025-04-26,NaN,1841.57,-32.0,True,False,NEGATIVE_CONVERSION_TIME
1,6867947d37953a251ea5eea5,6e103a09-c824-45c4-a29a-4be38d7bb2eb,ggeldert1@skyrock.com,Galina,Geldert,2025-05-24,WEB_ORGANIC,None,None,NaT,...,2025-03-04,"2460,79 €",2025-05-24,2025-03-04,NaN,2460.79,-81.0,True,False,NEGATIVE_CONVERSION_TIME
2,6867947d37953a251ea5eea6,95d9f881-6af6-47a3-a656-cba3135f7f2b,lmorales2@multiply.com,Lynn,Morales,2024-06-08,CAMPAIGN_003,CAMPAIGN_003,Selectividad_googleads,2024-11-03,...,2024-12-09,"2755,03 €",2024-06-08,2024-12-09,2028.94,2755.03,184.0,True,True,NO_ANOMALY
3,6867947d37953a251ea5eea7,18036b65-9742-4973-9486-4102725bd84e,cflanaghan3@yolasite.com,Courtnay,Flanaghan,2024-09-09,CAMPAIGN_008,CAMPAIGN_008,fp_meta,2025-01-19,...,2024-08-03,"953,44 €",2024-09-09,2024-08-03,2342.89,953.44,-37.0,True,False,NEGATIVE_CONVERSION_TIME
4,6867947d37953a251ea5eea8,b778e4f5-3ffb-42b4-be7c-79788c7f44a6,rpellingar4@google.com.hk,Rorie,Pellingar,2025-04-01,WEB_ORGANIC,None,None,NaT,...,2024-06-10,"1165,67 €",2025-04-01,2024-06-10,NaN,1165.67,-295.0,True,False,NEGATIVE_CONVERSION_TIME


In [3]:
# Select campaigns and date range
selected_campaigns = df["name"].dropna().unique().tolist()  # or manually define a subset
start_date = df["lead_created_at"].min()
end_date = df["lead_created_at"].max()

# Filtered DataFrame
filtered_df = df[
    (df["name"].isin(selected_campaigns)) &
    (df["lead_created_at"] >= pd.to_datetime(start_date)) &
    (df["lead_created_at"] <= pd.to_datetime(end_date))
].copy()

In [4]:
# Total Leads
total_leads = len(filtered_df)

# Valid conversions
valid_conversions = filtered_df["conversion_valid"].sum()

# Invalid conversions
invalid_df = filtered_df[(filtered_df["conversion_valid"] == False) & (filtered_df["converted"] == True)]
invalid_conversions = len(invalid_df)

print("Total Leads:", total_leads)
print("Valid Conversions:", valid_conversions)
print("Invalid Conversions:", invalid_conversions)

Total Leads: 728
Valid Conversions: 41
Invalid Conversions: 40


In [5]:
conversion_by_campaign = filtered_df.groupby("name")["converted"].sum().reset_index()

fig1 = px.bar(
    conversion_by_campaign,
    x="name",
    y="converted",
    title="Conversión por Campaña",
    labels={"name": "Campaña", "converted": "Convertidos"},
)
fig1.show()

In [6]:
fig2 = px.box(
    filtered_df,
    x="name",
    y="amount_float",
    title="Distribución de Monto Pagado por Campaña",
    labels={"name": "Campaña", "amount_float": "Monto (€)"},
)
fig2.show()

In [7]:
anomalies_df = filtered_df[
    (filtered_df["conversion_anomaly_type"] != "NO_ANOMALY") &
    (filtered_df["converted"] == True)
]

anomaly_counts = anomalies_df.groupby(
    ["name", "conversion_anomaly_type"]
).size().reset_index(name="count")

fig3 = px.bar(
    anomaly_counts,
    x="name",
    y="count",
    color="conversion_anomaly_type",
    title="Tipos de Anomalía por Campaña",
    labels={
        "name": "Campaña",
        "count": "Cantidad",
        "conversion_anomaly_type": "Tipo de Anomalía",
    },
)
fig3.show()

In [8]:
source_base_df = filtered_df.copy()
source_numerator_df = filtered_df[filtered_df["conversion_valid"] == True]

if "input_channel" in df.columns:
    source_conversion = (
        source_base_df.groupby("input_channel")["lead_id"].count().reset_index(name="total_leads")
        .merge(
            source_numerator_df.groupby("input_channel")["converted"].sum().reset_index(name="total_converted"),
            on="input_channel",
            how="left"
        )
    )
    source_conversion["conversion_rate"] = (
        source_conversion["total_converted"] / source_conversion["total_leads"] * 100
    )

    fig4 = px.bar(
        source_conversion,
        x="input_channel",
        y="conversion_rate",
        title="Tasa de Conversión por Fuente de Campaña",
        labels={"input_channel": "Fuente de Campaña", "conversion_rate": "Tasa de Conversión (%)"},
    )
    fig4.show()

In [9]:
campaign_base_df = filtered_df.copy()
campaign_numerator_df = campaign_base_df[campaign_base_df["conversion_valid"] == True]

conversion_rate_by_campaign = (
    campaign_base_df.groupby("name")["lead_id"].count().reset_index(name="total_leads")
    .merge(
        campaign_numerator_df.groupby("name")["converted"].sum().reset_index(name="total_converted"),
        on="name",
        how="left"
    )
)
conversion_rate_by_campaign["conversion_rate"] = (
    conversion_rate_by_campaign["total_converted"] / conversion_rate_by_campaign["total_leads"] * 100
)

fig5 = px.bar(
    conversion_rate_by_campaign,
    x="name",
    y="conversion_rate",
    title="Porcentaje de Conversión de Leads por Campaña",
    labels={"name": "Campaña", "conversion_rate": "Tasa de Conversión (%)"},
)
fig5.show()

In [10]:
converted_df = campaign_numerator_df[campaign_numerator_df["converted"] == True]

funnel_fig = go.Figure(go.Funnel(
    y=["Leads", "Convertidos"],
    x=[len(campaign_base_df), len(converted_df)],
    textinfo="value+percent previous"
))
funnel_fig.update_layout(title="Embudo de Conversión")
funnel_fig.show()

In [17]:
fig1.write_image("../output/fig1_conversion_by_campaign.png")
fig2.write_image("../output/fig2_boxplot_amount.png")
fig3.write_image("../output/fig3_anomalies.png")
fig4.write_image("../output/fig4_source_conversion.png")
fig5.write_image("../output/fig5_campaign_rate.png")
funnel_fig.write_image("../output/fig6_funnel.png")

In [ ]:
from fpdf import FPDF
import os

chart_dir = "../output/"
pdf_output_path = os.path.join(chart_dir, "reporte_conversion_campanas.pdf")

# PDF creation
pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=15)

def add_chart_with_text(title, image_filename, logic_text, interpretation_text):
    pdf.add_page()
    pdf.set_font("Arial", "B", 16)
    pdf.cell(0, 10, title, ln=True)
    pdf.ln(5)

    if os.path.exists(image_filename):
        pdf.image(image_filename, x=10, w=190)
        pdf.ln(5)
    else:
        pdf.set_font("Arial", "", 12)
        pdf.multi_cell(0, 10, f"[Imagen no encontrada: {image_filename}]")
    
    # Calculation logic
    pdf.set_font("Arial", "B", 12)
    pdf.cell(0, 10, "Lógica de cálculo:", ln=True)
    pdf.set_font("Arial", "", 12)
    pdf.multi_cell(0, 10, logic_text)

    # Interpretation
    pdf.ln(2)
    pdf.set_font("Arial", "B", 12)
    pdf.cell(0, 10, "Interpretación:", ln=True)
    pdf.set_font("Arial", "", 12)
    pdf.multi_cell(0, 10, interpretation_text)

# cover page
pdf.add_page()
pdf.set_font("Arial", "B", 20)
pdf.cell(0, 10, "Reporte de Conversión de Campañas", ln=True, align="C")
pdf.ln(10)
pdf.set_font("Arial", "", 14)
pdf.multi_cell(0, 10, "Este informe presenta un análisis visual y descriptivo de los principales indicadores relacionados con las campañas de marketing digital.")

# main metrics
pdf.ln(10)
pdf.set_font("Arial", "B", 12)
pdf.cell(0, 10, "Métricas generales:", ln=True)
pdf.set_font("Arial", "", 12)
pdf.cell(0, 10, "Total de Leads: 728", ln=True)
pdf.cell(0, 10, "Conversiones válidas: 41", ln=True)
pdf.cell(0, 10, "Conversiones inválidas: 40", ln=True)

# Graphic content
charts = [
    {
        "filename": "fig1_conversion_by_campaign.png",
        "title": "Conversión por Campaña",
        "logic": "Se agrupan los datos por nombre de campaña y se suman las conversiones (converted == True) para cada una.",
        "interpretation": "Este gráfico muestra cuántos leads fueron convertidos por cada campaña. 'Selectividad_Meta' lidera en conversiones, indicando un buen rendimiento."
    },
    {
        "filename": "fig2_boxplot_amount.png",
        "title": "Distribución de Monto Pagado por Campaña",
        "logic": "Se utiliza un gráfico de caja (boxplot) para mostrar la distribución del campo 'amount_float' agrupado por campaña.",
        "interpretation": "Se observan campañas con gran dispersión de montos pagados, como 'opos_meta' y 'Selectividad_googleads', lo cual podría reflejar estrategias de precios diversas."
    },
    {
        "filename": "fig3_anomalies.png",
        "title": "Tipos de Anomalía por Campaña",
        "logic": "Se filtran leads convertidos con anomalías (conversion_anomaly_type != 'NO_ANOMALY') y se cuenta por tipo y campaña.",
        "interpretation": "Las anomalías más comunes están relacionadas con el tiempo negativo de conversión. Es importante investigarlas para mejorar los procesos."
    },
    {
        "filename": "fig4_source_conversion.png",
        "title": "Tasa de Conversión por Fuente de Campaña",
        "logic": "Para cada fuente (input_channel), se calcula la tasa: total convertidos / total leads * 100.",
        "interpretation": "Las fuentes 'CAMPAIGN_002' y 'CAMPAIGN_004' presentan tasas más altas de conversión, lo cual sugiere mayor eficacia de esos canales."
    },
    {
        "filename": "fig5_campaign_rate.png",
        "title": "Porcentaje de Conversión de Leads por Campaña",
        "logic": "Se agrupa por campaña y se calcula la tasa: convertidos válidos / total de leads * 100.",
        "interpretation": "'Selectividad_Meta' destaca por su alta tasa de conversión, seguida de 'opos_TikTok' y 'opos_meta'."
    },
    {
        "filename": "fig6_funnel.png",
        "title": "Embudo de Conversión",
        "logic": "Se compara el total de leads con el total de leads convertidos para representar el embudo de conversión.",
        "interpretation": "De 728 leads, solo 41 se convirtieron, lo que representa una tasa de conversión general del 6%. Esto indica espacio para optimización del embudo."
    }
]

# Add each chart with its text
for chart in charts:
    image_path = os.path.join(chart_dir, chart["filename"])
    add_chart_with_text(chart["title"], image_path, chart["logic"], chart["interpretation"])

# save the PDF
pdf.output(pdf_output_path)
print(f"PDF sucessfuly created: {pdf_output_path}")

✅ PDF gerado com sucesso: ../output/reporte_conversion_campanas.pdf
